In [2]:
!pip install cleantext

In [7]:
help(clean)

Help on function clean in module cleantext.cleantext:

clean(text: str, clean_all: bool = True, extra_spaces: bool = False, stemming: bool = False, stopwords: bool = False, lowercase: bool = False, numbers: bool = False, punct: bool = False, reg: str = '', reg_replace: str = '', stp_lang: str = 'english') -> str
    Given a raw string, return cleaned text
    
    :param text: Input text to clean
    :param clean_all: Execute all cleaning operations
    :param extra_spaces: Remove extra white spaces
    :param stemming: Stem the words
    :param stopwords: Remove stop words
    :param lowercase: Convert to lowercase
    :param numbers: Remove all digits
    :param punct: Remove all punctuations
    :param reg: Regular expression for removing or replacing
    :param reg_replace: Replace the part with regular expression(reg)
    :param stp_lang: Language for stop words
    :return: Cleaned text



In [6]:
import hazm
from cleantext import clean
import re

def cleanhtml(raw_html):
    cleanr = re.compile('<.*?>')
    cleantext = re.sub(cleanr, '', raw_html)
    return cleantext


def cleaning(text):
    text = text.strip()
    
    # regular cleaning
    text = clean(text,
        fix_unicode=True,
        to_ascii=False,
        lowercase=True,
        no_line_breaks=True,
        no_urls=True,
        no_emails=True,
        no_phone_numbers=True,
        no_numbers=False,
        no_digits=False,
        no_currency_symbols=True,
        no_punct=False,
        replace_with_url="",
        replace_with_email="",
        replace_with_phone_number="",
        replace_with_number="",
        replace_with_digit="0",
        replace_with_currency_symbol="",
    )

    # cleaning htmls
    text = cleanhtml(text)
    
    # normalizing
    normalizer = hazm.Normalizer()
    text = normalizer.normalize(text)
    
    # removing wierd patterns
    wierd_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        u"\U0001f926-\U0001f937"
        u'\U00010000-\U0010ffff'
        u"\u200d"
        u"\u2640-\u2642"
        u"\u2600-\u2B55"
        u"\u23cf"
        u"\u23e9"
        u"\u231a"
        u"\u3030"
        u"\ufe0f"
        u"\u2069"
        u"\u2066"
        # u"\u200c"
        u"\u2068"
        u"\u2067"
        "]+", flags=re.UNICODE)
    
    text = wierd_pattern.sub(r'', text)
    
    # removing extra spaces, hashtags
    text = re.sub("#", "", text)
    text = re.sub("\s+", " ", text)
    
    return text

with open('dataset.txt', 'r', encoding='utf-8') as f: #open the file
    total_dialogues = []
    for line in f:
        new_dialogue = []
        dialogue = line.strip().split('$')
        for utter in dialogue:
#             token_list = tokenizer.tokenize(utter.strip().replace(pre_quote, quotes[1]))
#             token_list = process_token_list(token_list)
#             text = tokenizer.convert_tokens_to_string(token_list)
            utter = cleaning(utter)
            new_dialogue.append(utter)
        
        print(new_dialogue)
        total_dialogues.append(new_dialogue)
    
    train_frac = 0.8
    train_utter_num = 0
    valid_utter_num = 0
    train_dialogues = total_dialogues[:int(len(total_dialogues)*train_frac)]
    valid_dialogues = total_dialogues[int(len(total_dialogues)*train_frac):]
    
    for dialogue in train_dialogues:
        train_utter_num += len(dialogue)
        
    for dialogue in valid_dialogues:
        valid_utter_num += len(dialogue)

    print(train_utter_num)
    print(train_dialogues)
    print(valid_dialogues)
    print(valid_utter_num)

TypeError: clean() got an unexpected keyword argument 'no_line_breaks'

In [ ]:
open(stop_words_path, encoding='utf-8').read()

In [ ]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import f1_score
from sklearn.utils import shuffle

import hazm
from cleantext import clean

import plotly.express as px
import plotly.graph_objects as go

from tqdm.notebook import tqdm

import os
import re
import json
import copy
import collections

In [ ]:
data = pd.read_csv('dataset.txt', encoding='utf-8', header=None, delimiter ='$', names=['person1','person2'])

In [ ]:
# print data information
print('data information')
# print(data.info(), '\n')

# print missing values information
print('missing values stats')
print(data.isnull().sum(), '\n')


In [ ]:
def cleanhtml(raw_html):
    cleanr = re.compile('<.*?>')
    cleantext = re.sub(cleanr, '', raw_html)
    return cleantext


def cleaning(text):
    text = text.strip()
    
    # regular cleaning
    text = clean(text,
        fix_unicode=True,
        to_ascii=False,
        lower=True,
        no_line_breaks=True,
        no_urls=True,
        no_emails=True,
        no_phone_numbers=True,
        no_numbers=False,
        no_digits=False,
        no_currency_symbols=True,
        no_punct=False,
        replace_with_url="",
        replace_with_email="",
        replace_with_phone_number="",
        replace_with_number="",
        replace_with_digit="0",
        replace_with_currency_symbol="",
    )

    # cleaning htmls
    text = cleanhtml(text)
    
    # normalizing
    normalizer = hazm.Normalizer()
    text = normalizer.normalize(text)
    
    # removing wierd patterns
    wierd_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        u"\U0001f926-\U0001f937"
        u'\U00010000-\U0010ffff'
        u"\u200d"
        u"\u2640-\u2642"
        u"\u2600-\u2B55"
        u"\u23cf"
        u"\u23e9"
        u"\u231a"
        u"\u3030"
        u"\ufe0f"
        u"\u2069"
        u"\u2066"
        # u"\u200c"
        u"\u2068"
        u"\u2067"
        "]+", flags=re.UNICODE)
    
    text = wierd_pattern.sub(r'', text)
    
    # removing extra spaces, hashtags
    text = re.sub("#", "", text)
    text = re.sub("\s+", " ", text)
    
    return text

In [ ]:
data['cleaned_person1'] = data['person1'].apply(cleaning)
data['cleaned_person2'] = data['person2'].apply(cleaning)

In [ ]:
data.head(10)

In [ ]:
data = data[['cleaned_person1', 'cleaned_person2']]

In [ ]:
data.columns = ['person1', 'person2']

In [ ]:
!wget https://cdn-lfs.huggingface.co/m3hrdadfi/gpt2-persian-qa.tar.gz

In [ ]:
from transformers import AutoConfig, AutoTokenizer, AutoModel

# v3.0
model_name_or_path = "HooshvareLab/bert-fa-base-uncased"
config = AutoConfig.from_pretrained(model_name_or_path)
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path)

model = AutoModel.from_pretrained(model_name_or_path)

In [ ]:
from transformers import AutoConfig, AutoTokenizer, AutoModelWithLMHead, OpenAIGPTDoubleHeadsModel, OpenAIGPTTokenizer
from transformers import pipeline, AutoTokenizer, GPT2LMHeadModel

# model_name_or_path = "HooshvareLab/bert-fa-base-uncased"
# config = AutoConfig.from_pretrained(model_name_or_path)
tokenizer = AutoTokenizer.from_pretrained('flax-community/gpt2-medium-persian')

model = GPT2LMHeadModel.from_pretrained('flax-community/gpt2-medium-persian')

In [ ]:
SPECIAL_TOKENS = {'bos_token':'[bos]', 'eos_token':'[eos]', 'speaker1_token':'[speaker1]', 'speaker2':'[speaker2]', 'pad_token':'[pad]'}

# We can add these special tokens to the vocabulary and the embeddings of the model:
tokenizer.add_special_tokens(SPECIAL_TOKENS)
model.resize_token_embeddings(len(tokenizer))

In [ ]:
from transformers import AutoTokenizer
from transformers import GPT2LMHeadModel

model_name_or_path = "m3hrdadfi/gpt2-persian-qa"
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path)
model = GPT2LMHeadModel.from_pretrained(model_name_or_path)